In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
#mostrar bancos de dados e tabelas
spark.sql("show databases").show()
#criar banco de dados
spark.sql("create database desp")
spark.sql("use desp").show()

In [ ]:
#criar tabela gerenciada
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv("dados/despachantes.csv", header=False, schema=arqschema)
despachantes.write.saveAsTable("Despachantes")

In [ ]:
#mostrar que a tabela existe
spark.sql("select * from despachantes").show()

In [ ]:
#mostra tabela
spark.sql("show tables").show()

In [ ]:
#mudar banco de dados
spark.sql("use default").show()

In [ ]:
#executa novamente e mostrar que da erro
spark.sql("select * from despachantes").show()

In [ ]:
#voltar ao nosso banco de dados
spark.sql("use desp").show()

In [ ]:
# overwrite e append
despachantes.write.mode("overwrite").saveAsTable("Despachantes")

In [ ]:
#teste de pesistência
from pyspark.sql import SparkSession
spark.sql("use desp").show()
#mostrar que a tabela ainda existe
spark.sql("select * from despachantes").show()
despachantes.show()

In [ ]:
#o resultado de uma consulta sem um show gera um dataframe
despachantes = spark.sql("select * from despachantes")
despachantes.show()

In [ ]:
#criar tabela não gerenciada
#salvamos novamente no formato parquet, em outro diretorio
despachantes.write.format("parquet").save("dados/desparquet") 
#informar o caminho 
despachantes.write.option("path", "dados/desparquet").saveAsTable("Despachantes_ng")

In [ ]:
#como saber se uma tabela é gerenciada ou não?
#podemos observar que despachantes não mostra o caminho
spark.sql("show create table Despachantes").show(truncate=False)
#despachantes ng mostra, indicando que é não gerenciada
spark.sql("show create table Despachantes_ng").show(truncate=False)

In [ ]:
#outra forma:
spark.catalog.listTables()

In [ ]:
##views

despachantes.createOrReplaceTempView("Despachantes")
despachantes.createOrReplaceGlobalTempView("Despachantes")

In [ ]:
#consultas
from pyspark.sql import functions as Func
from pyspark.sql.functions import sum

In [ ]:
#mostrar a tabela
spark.sql("Select * from Despachantes").show()
despachantes.show()

In [ ]:
#mostrar certas colunas
spark.sql("Select nome,vendas from Despachantes").show()
despachantes.select("nome","vendas").show()

In [ ]:
#condição lógica
spark.sql("Select nome,vendas from Despachantes where vendas > 20").show()
despachantes.select("id","nome","vendas").where(Func.col("vendas") > 20).show()

In [ ]:
spark.sql("Select cidade,sum(vendas) from Despachantes group by cidade order by 2 desc").show()
despachantes.groupBy("cidade").agg(sum("vendas")).orderBy(Func.col("sum(vendas)").desc()).show()

In [ ]:
recschema = "idrec INT, datarec STRING, iddesp INT"
reclamacoes = spark.read.csv("dados/reclamacoes.csv", header=False, schema=recschema)
reclamacoes.write.saveAsTable("reclamacoes")

In [ ]:
#inner join
spark.sql("select reclamacoes.*, despachantes.nome from despachantes inner join reclamacoes  on (despachantes.id = reclamacoes.iddesp)").show()


In [ ]:
#righ join deve trazer o mesmo resultado, pois todas as reclamações tem um despachante
spark.sql("select reclamacoes.*, despachantes.nome from despachantes right join reclamacoes on (despachantes.id = reclamacoes.iddesp)").show()

In [ ]:
#um left join traz mais colunas e campos nulos, pois alguns despachantes não tem reclações
spark.sql("select reclamacoes.*, despachantes.nome from despachantes left join reclamacoes  on (despachantes.id = reclamacoes.iddesp)").show()


In [ ]:
#inner join
despachantes.join(reclamacoes,despachantes.id == reclamacoes.iddesp, "inner").select("idrec","datarec","iddesp","nome").show()


In [ ]:
#right
despachantes.join(reclamacoes,despachantes.id == reclamacoes.iddesp, "right").select("idrec","datarec","iddesp","nome").show()


In [ ]:

#left
despachantes.join(reclamacoes,despachantes.id == reclamacoes.iddesp, "right").select("idrec","datarec","iddesp","nome").show()


In [ ]:

##spark-sql
show databases;
use desp;
show tables;

Select * from Despachantes;

Select nome,vendas from Despachantes;

#condição lógica
Select nome,vendas from Despachantes where vendas > 20;

#agrupar
Select cidade,sum(vendas) from Despachantes group by cidade order by 2 desc;

join
select reclamacoes.*, despachantes.nome from despachantes inner join reclamacoes  on (despachantes.id = reclamacoes.iddesp);